# AutoscoutDataAnalysis

## Choice and tests on different models - Lazy predict

## Authors : 
- Mozerski Alex
- Fraile Diego
- Spycher Damian

#### Projet semestre d'été 2024
version 1.0

### Imports

In [62]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import networkx as nx
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder

### Constant

In [63]:
DATA_FILE = os.path.join(os.getcwd(), 'cleaned_cars.csv')

### Data Loading

In [64]:
df = pd.read_csv(DATA_FILE, header=[0], index_col=0)

### Overview of the dataset

In [65]:
df.head()

,brand,model,price,mileage,fuel_type,color,gearbox,power,engine_size,seller,body_type,doors,seats,drivetrain,emission_class,condition,upholstery,upholstery_color,year,country
url,,,,,,,,,,,,,,,,,,,,
https://www.autoscout24.com/offers/alfa-romeo-159-berlina-2-0-jtdm-super-diesel-red-7140b39f-39f8-4707-833e-3ac1344299b2,alfa romeo,159,5500.00,233685.00,diesel,red,manual,136.00,1956.00,dealer,sedan,4.00,5.00,front,134.00,used,grey,NaN,2011.00,it
https://www.autoscout24.com/offers/alfa-romeo-159-1-8-tbi-elegante-gasoline-red-edd10d3e-53b3-4201-9d2b-73216507df12,alfa romeo,159,13990.00,134000.00,gasoline,red,manual,200.00,1742.00,dealer,sedan,4.00,5.00,front,NaN,used,NaN,NaN,2010.00,es
https://www.autoscout24.com/offers/renault-others-dauphin-gasoline-yellow-762bffc3-b9b0-4c4b-b14d-f066356eb926,renault,unknown,2300.00,27000.00,gasoline,yellow,manual,80.00,800.00,dealer,sedan,NaN,NaN,NaN,NaN,used,NaN,alcantara,1963.00,be
https://www.autoscout24.com/offers/opel-antara-2-4-16v-temptation-trekhaak-gasoline-black-09934fcb-5180-47e3-9c90-e1cdb6215977,opel,antara,3499.00,212575.00,gasoline,black,manual,140.00,2405.00,dealer,off-road/pick-up,5.00,5.00,4wd,229.00,used,NaN,metallic,2008.00,nl
https://www.autoscout24.com/offers/alfa-romeo-giulia-2-0-turbo-automatik-ti-q4-leder-navi-kamera-gasoline-white-7e452f2c-a446-4b1d-a072-20462e8d2d2b,alfa romeo,giulia,39800.00,7100.00,gasoline,white,automatic,280.00,1995.00,dealer,sedan,4.00,4.00,front,NaN,employee's car,full leather,black,2023.00,de


In [66]:
print("Missing data in each column:")

print(df.isnull().sum())

Missing data in each column:
brand                  0
model                  0
price                  0
mileage              181
fuel_type             26
color                110
gearbox              401
power                236
engine_size         1626
seller                 0
body_type              0
doors               4264
seats               1329
drivetrain          3886
emission_class      6048
condition              0
upholstery          4414
upholstery_color    3964
year                 718
country                0
dtype: int64


In [67]:
# Remove prices above 1'000'000
df = df[df['price'] < 1000000]

In [68]:
df = df.select_dtypes(include = ['float64', 'int64'])

In [69]:
#df = df.dropna()

In [70]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor
import pandas as pd




## Lazy predict

In [71]:
predict_col= "price"
data_cols= df.columns[df.columns != predict_col]
data_cols, predict_col

(Index(['mileage', 'power', 'engine_size', 'doors', 'seats', 'emission_class',
        'year'],
       dtype='object'),
 'price')

In [72]:
X_train, X_test, y_train, y_test = train_test_split(df[data_cols],df[predict_col] , test_size=0.2, shuffle=False)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(12244, 7) (3061, 7) (12244,) (3061,)


In [73]:
clf = LazyRegressor(verbose=0,ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print(models)

100%|██████████| 42/42 [00:54<00:00,  1.31s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000212 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1085
[LightGBM] [Info] Number of data points in the train set: 12244, number of used features: 7
[LightGBM] [Info] Start training from score 68607.956877
                               Adjusted R-Squared  R-Squared        RMSE  \
Model                                                                      
RANSACRegressor                              0.53       0.53    11219.71   
GammaRegressor                               0.05       0.05    15871.53   
SVR                                          0.04       0.04    15979.07   
NuSVR                                       -0.43      -0.42    19454.77   
LinearSVR                                   -0.74      -0.74    21497.64   
MLPRegressor                                -1.08      -1.08    23506.58   
XGBRegressor                                -1.

In [74]:
# check if train, test has some NaN values
print(X_train.isnull().sum())
print(X_test.isnull().sum())
print(y_train.isnull().sum())
print(y_test.isnull().sum())

mileage            138
power              204
engine_size       1061
doors             3607
seats             1144
emission_class    4834
year               506
dtype: int64
mileage             41
power               19
engine_size        522
doors              598
seats              158
emission_class    1140
year               203
dtype: int64
0
0


In [75]:
from sklearn.experimental import enable_hist_gradient_boosting  # Explicitly require this experimental feature
from sklearn.ensemble import HistGradientBoostingRegressor

# Initialize and fit the model
reg = HistGradientBoostingRegressor().fit(X_train, y_train)

# Predict on the test data
y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(mse)

814711228.8063864


In [76]:
from sklearn.linear_model import GammaRegressor

reg = GammaRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(mse)

ValueError: Input X contains NaN.
GammaRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values